In [2]:
import pymongo
from sklearn.ensemble import IsolationForest
import numpy as np
import pandas as pd

# MongoDB connection details
#MONGO_URI = "" Colocar connection string
DATABASE = "gastospublicos"
COLLECTION_NAME = 'slv_notasfiscais'


In [29]:
client = pymongo.MongoClient(MONGO_URI)
db = client[DATABASE]
collection = db[COLLECTION_NAME]

# Extract relevant features for anomaly detection (ensuring a 2D array)
data = [
    [
        int(document["CNPJ EMITENTE"]),
        int(document["CNPJ DESTINATÁRIO"]),
        int(document["VALOR NOTA FISCAL"])
    ]
    for document in collection.find()
]

# Convert data to a NumPy array for better shape handling
data = np.array(data)

# Debugging: Print shape and data type
print(f"Data shape: {data.shape}")
print(f"Data: {data[:5]}")  # Print first 5 entries to verify the content

# Ensure data is in 2D array
if len(data.shape) == 1:
    data = data.reshape(-1, 3)

# Debugging: Print the reshaped data
print(f"Reshaped Data shape: {data.shape}")

# Train the Isolation Forest model
model = IsolationForest(contamination="auto")  # Adjust contamination as needed
model.fit(data)

# Predict anomaly scores
anomaly_scores = model.decision_function(data)

# Identify anomalies based on a threshold
threshold = 0  # Adjust threshold as needed
anomalies = [i for i, score in enumerate(anomaly_scores) if score < threshold]

# Print the anomalous documents
print("Anomalous documents:")
for index in anomalies:
    document = collection.find().skip(index).limit(1)[0]  # Retrieve the document at the specific index
    print(document)

# Close the MongoDB connection
client.close()


KeyError: 'CNPJ EMITENTE'

In [3]:
client = pymongo.MongoClient(MONGO_URI)
db = client[DATABASE]
collection = db[COLLECTION_NAME]

features = ["VALOR NOTA FISCAL", # Replace/add features as needed.
            'CPF/CNPJ Emitente',
            'UF EMITENTE',
            'CNPJ DESTINATÁRIO',
            'DATA/HORA EVENTO MAIS RECENTE',
            'UF DESTINATÁRIO'
           ]
data_from_mongo = collection.find({}, {feature: 1 for feature in features}) # Project only the required features

# Convert to DataFrame
df = pd.DataFrame(list(data_from_mongo))




: 

In [ ]:
# 2. Data Preprocessing and Feature Engineering (Same as before):

# Select relevant numeric features
numeric_features = ['VALOR NOTA FISCAL', # List your numeric features
                    ]

data = df[numeric_features].copy()

# Handle missing values (if any)
data.fillna(data.mean(), inplace=True)  # Or use another strategy

# Feature scaling
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

# 3. Isolation Forest Model (Same as before):

model = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
model.fit(scaled_data)


# 4. Add Anomaly Scores and Predictions back to the DataFrame:

df['anomaly_score'] = model.decision_function(scaled_data)
df['anomaly'] = model.predict(scaled_data)



# 5.  Analysis and Investigation (Similar as before, but with MongoDB integration):

anomalies = df[df['anomaly'] == -1]

# ... (Analysis as before: grouping, analyzing distributions, etc.)


# Example: Update the MongoDB collection with anomaly flags:

#for index, row in anomalies.iterrows():
 #   collection.update_one({'_id': row['_id']}, {'$set': {'anomaly': True}}) # Mark as anomaly in MongoDB

# Or store anomaly results in a separate collection


# 6. Refinement and Iteration:  (As before)